In [ ]:
import numpy as np
import cv2
from skimage import io
from skimage import morphology
import random
import math

In [ ]:
# This field contains all img processing to find the lines in the field
def skeletonize(grayscale, threshold=127):
    size = np.size(grayscale)
    skel = np.zeros(grayscale.shape,np.uint8)

    ret,img = cv2.threshold(grayscale,threshold,255,0)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    done = False

    while(not done):
        eroded = cv2.erode(grayscale,element)
        temp = cv2.dilate(eroded,element)
        temp = cv2.subtract(grayscale,temp)
        skel = cv2.bitwise_or(skel,temp)
        grayscale = eroded.copy()

        zeros = size - cv2.countNonZero(grayscale)
        if zeros==size:
            done = True
        
    return np.uint8(skel)

def threshold(img, percentile=95):
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    retval, threshold = cv2.threshold(img, np.percentile(img, percentile), 255, cv2.THRESH_BINARY)
    return np.uint8(threshold)

def grayscale(img):
    return np.uint8(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))

def removeNonLineObjects(img, iterations=5, min_size=150, max_objects=10):    
    try:
        for i in range(0,max_objects):
            kernel = np.ones((3,3), np.uint8)
            remImg = cv2.erode(img, kernel, iterations=iterations) #Removes (thin) lines
            remImg = removeSmallComponents(remImg, min_size=min_size)
            mask = np.zeros((img.shape[0]+2, img.shape[1]+2), np.uint8)
            nonzero = np.nonzero(remImg)
            
            if len(nonzero) > 0 and len(nonzero[0]) > 0:
                cv2.floodFill(img, mask, (nonzero[1][0],nonzero[0][0]), 0);
            else:
                #print("No resulting non-lines found in iteration {0}".format(i))
                break
    except:
        #print("Could not remove non line objects")
        pass
    return img

def removeNonLinesUsingBoundingBox(img, fillThreshold = 0.5, ratioThreshold = 0.1):    
    img,contours,hierarchy = cv2.findContours(img, 1, 2)
    rects = []
    for cnt in contours:
        rect = get_min_area_rect(img, cnt)
        rects.append(rect)
    
    mask = np.zeros(img.shape[:2], dtype="uint8")

    # Go through all objects in img
    for i in range(0, len(rects)):
        rect = rects[i]

        # Calculate ratio, fill
        fill = 1 - len(rect[rect <= 0]) / (rect.shape[0]*rect.shape[1])
        ratio = min(rect.shape[0]/rect.shape[1],rect.shape[1]/rect.shape[0])

        if fill > fillThreshold and ratio > ratioThreshold:
            cv2.drawContours(mask, contours, i, 255, -1)
            #print("Removing object with fill {0}, ratio {1}".format(fill, ratio))
    
    return np.uint8(img - mask)

def get_min_area_rect(img, contour):
    # get rotated rect of contour and split into components
    center, size, angle = cv2.minAreaRect(contour)

    # not sure why this is needed, see 
    # http://felix.abecassis.me/2011/10/opencv-rotation-deskewing/
    if angle < -45.0:
        angle += 90.0
        width, height = size[0], size[1]
        size = (height, width)

    M = cv2.getRotationMatrix2D(center, angle, 1.0)

    # rotate the entire image around the center of the parking cell by the
    # angle of the rotated rect
    # codereview: not sure why it was necessary to swap width and height here,
    # probably related to the fact that we did angle += 90 earlier
    imgWidth, imgHeight = (img.shape[0], img.shape[1])
    rotated = cv2.warpAffine(img, M, (imgHeight, imgWidth), flags=cv2.INTER_CUBIC)

    # extract the rect after rotation has been done
    sizeInt = (np.int0(size[0]), np.int0(size[1]))
    uprightRect = cv2.getRectSubPix(rotated, sizeInt, center)
    return uprightRect

def removeSmallComponents(img, min_size = 150):
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=8)
    sizes = stats[1:, -1]; nb_components = nb_components - 1

    # return img
    img = np.zeros((output.shape))
    
    #for every component in the image, you keep it only if it's above min_size
    for i in range(0, nb_components):
        if sizes[i] >= min_size:
            img[output == i + 1] = 255
            
    return np.uint8(img)

def processImageToFieldLines(img):
    # Grayscale
    img = grayscale(frame)

    # Threshold
    img = threshold(img, 95)

    # Remove small components
    img = removeSmallComponents(img, min_size=150)
    
    # Remove non line objects
    img = removeNonLineObjects(img, iterations=5, min_size=100)

    # Remove small components
    img = removeSmallComponents(img, min_size=150)
    
    # Remove non line objects
    img = removeNonLinesUsingBoundingBox(img, fillThreshold = 0.5, ratioThreshold = 0.2)

    # Remove small components
    img = removeSmallComponents(img, min_size=150)
    
    # Skeletonize, so our samples will be on the center lines
    img = skeletonize(np.uint8(img))

    # Remove small components
    img = removeSmallComponents(img, min_size=3)
    
    return np.uint8(img)

In [ ]:
# This field contains all correction logic
def measureSamples(img, n=500):
    nonzero = np.nonzero(img)
    indices = random.sample(range(0,len(nonzero[0])), n)
    center = (img.shape[0]/2, img.shape[1]/2)

    results = []

    for i in indices:
        dist = distance(nonzero[0][i] - center[0], nonzero[1][i]-center[1])
        distComp = distanceCompensation(dist)
        an = angle(nonzero[0][i] - center[0], nonzero[1][i]-center[1], t="radian")
        results.append({"distance":distComp,"angle":an})

    return results

def distance(x, y):
    return math.sqrt(math.pow(x,2)+math.pow(y,2))

def distanceCompensation(x, radius=275):
    x = abs(x)/radius
#     return (math.pow(x,3)+3*x)/4
    return (math.pow(x,3)+1.5*x)/2.5

def angle(x,y,t="radian"):
    res = np.arctan2(x,y)
    
    if res < 0:
        res += 2*math.pi
    
    if t=="degree":
        return res/(2*math.pi)*360
    else:
        return res
    
def samplesDistortionCorrection(samples, w=500, h=500):
    img = np.zeros((w,h),np.uint8)
    correctedSamples = []

    for sample in samples:
        dist = sample["distance"]
        an = sample["angle"]
        x = 1+(math.cos(math.pi/2 - an))*dist
        y = 1+(math.sin(math.pi/2 - an))*dist
        xImg = int((x*(w/2)))
        yImg = int((y*(h/2)))
        correctedSamples.append((dist,an)) #TODO or x, y?
        img[xImg,yImg] = 255
        
    return img, correctedSamples

In [ ]:
# This field contains all localization logic
def readFieldBitmap():
    field = cv2.imread('field.png',0)
    return field

def generateParticles(w, h, n=50):
    """ With replace(!)
    """
    particles = []
    for i in range(0,n):
        particles.append((
            random.randint(0, w-1), 
            random.randint(0, h-1), 
            random.random()*2*math.pi, 
            0.005,
            1.0
        ))
    return particles

def generateParticleReplacement(locationParticles, 
                                w, 
                                h, 
                                totalRandomFactor = 0.2, 
                                locationRandomFactor=0.05, 
                                angleRandomFactor=0.1):
    if random.random() > totalRandomFactor:
        existing = random.sample(locationParticles, min(len(locationParticles), 25))
        for particle in existing:
            if particle is not None:
                return (
                    min(w-1, max(0,particle[0]+(((random.random()-1)/2)*w) * locationRandomFactor)),
                    min(h-1, max(0,particle[1]+(((random.random()-1)/2)*w) * locationRandomFactor)),
                    particle[2] + (((random.random()-1)/2) * angleRandomFactor),
                    particle[3],
                    particle[4]
                )
            else:
                # We randomly picked a dead particle. My bad!
                continue
    
    # In the unlikely situation we couldn't clone any existing particle
    # Or, totalrandom decided to spawn a random particle
    # Generate a new one.
    return (
            random.randint(0, w-1), 
            random.randint(0, h-1), 
            random.random()*2*math.pi, 
            0.005,
            1.0
        )

def calculateCloseness(fieldSamples, lineSamples, n=10, lineFieldOfView=0.2):
    # Scale fieldSamples
    for i in range(0, len(fieldSamples)):
        fieldSamples[i] = (fieldSamples[i][0] / lineFieldOfView, fieldSamples[i][1])
    
    # Create short, long list
    if len(lineSamples) > len(fieldSamples):
        sh = fieldSamples
        lg = lineSamples
    else:
        sh = lineSamples
        lg = fieldSamples
        
    # Lists of distances
    shs = []
    for s in sh:
        shs.append(s[0])
    lgs = []
    for l in lg:
        lgs.append(l[0])
        
    # Get n most similar distances
    distances = []
    for s in shs:
        ms = 1000
        msi = 0
        for i in range(0, len(lgs)):
            l = lgs[i]
            if abs(s - l) < ms:
                ms = abs(s - l)
                msi = i
        del lgs[msi] # Only choose once
        distances.append(ms)
    distances.sort()
    distances = distances[:n]
    
    # Recalculate distances to "closeness" value
    closeness = max(0, min(1, sum(distances) / 5*n))
    return closeness

def updateParticle(particle, lineSamples, field, fieldSamples=10, vicinity=25):
    # Particle = (x, y, direction, velocity)
    # Move particle, with speed, in direction
    dist = particle[3]*((field.shape[0]+field.shape[1])/2)
    x = (math.cos(math.pi/2 - particle[2]))*dist
    y = (math.sin(math.pi/2 - particle[2]))*dist
    particle = (particle[0] + x, particle[1] + y, particle[2], particle[3], particle[4])
    
    # If particle out of bounds, kill
    if (particle[0] < 0 
            or particle[0] > field.shape[0]-1 
            or particle[1] < 0 
            or particle[1] > field.shape[1]-1):
        return None # Dead particle :( (:'))
    
    # Take n random samples from map in near vicinity
    subField = field[int(particle[0])-vicinity:int(particle[0])+vicinity,int(particle[1])-vicinity:int(particle[1])+vicinity]
    nonzero = np.nonzero(subField)
    if len(nonzero) > 1 and len(nonzero[0]) > 0:
        indices = random.sample(range(0,len(nonzero[0])), min(len(nonzero[0]), fieldSamples))
        subfieldSamples = []
        for i in indices:
            # Process each subfieldSample
            subfieldSample = (nonzero[0][i], nonzero[1][i])

            # Calculate distance, angle from subfieldSample to particle
            dist = distance(abs(subfieldSample[0] - particle[0])/field.shape[0], abs(subfieldSample[1] - particle[1])/field.shape[1])
            an = angle(subfieldSample[0] - particle[0],subfieldSample[1] - particle[1])

            subfieldSamples.append((dist, an))

        # Compare fieldSample distances+angles to lineSamples
        # Should be angle agnostic, meaning that angle within subfieldSamples and lineSamples is important.
        # But angle between subfieldSamples and lineSamples is not important. It can be rotated
        closeness = calculateCloseness(subfieldSamples, lineSamples)
        
        # Calculate closeness value
        particle = (particle[0], particle[1], particle[2], particle[3], closeness) # TODO new closeness value
    
    # If close, update direction and speed
    if particle[4] < 0.005:
#         direction = particle[2]  # TODO alter based on better direction(how?)
        particle = (particle[0], particle[1], particle[2], particle[3] * 0.5, particle[4])
    elif particle[4] >= 0.007:
        particle = (particle[0], particle[1], particle[2], min(0.05, particle[3] * 1.1), particle[4])
    
    # Return new particle
    return particle

field = readFieldBitmap()
locationParticles = generateParticles(field.shape[0], field.shape[1], n=100)

In [ ]:
# # Only do first frame
# cap = cv2.VideoCapture('vid.mpg')

# # Read frame 1
# ret, frame = cap.read()
# img = processImageToFieldLines(frame)
    
# # Sample, measure distances
# results = measureSamples(img, 300)
# crsImg, crs = samplesDistortionCorrection(results)

# # Show
# cv2.imshow('corrected line',crsImg)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.waitKey(1)

In [ ]:
# Do entire video
cap = cv2.VideoCapture('vid.mpg')
fails = 0

while(cap.isOpened()):
    if fails > 100:
        break
    
    try:
        ret, frame = cap.read()
        img = processImageToFieldLines(frame)
    except:
        print("Could not process frame")
        fails += 1
        continue
    
    try:
        results = measureSamples(img, 100)
        img, crs = samplesDistortionCorrection(results)
    except:
        print("Could not process distortion")
        fails += 1
        continue
        
    try:
        dispField = np.copy(field)

        # update location particles
        for i in range(0,len(locationParticles)):
            locationParticles[i] = updateParticle(locationParticles[i], crs, field, fieldSamples=25, vicinity=40)

        # Remove None particles
        newLocationParticles = []
        for particle in locationParticles:
            if random.random() < 0.01:
                # Randomly replace this particle
                newParticle = generateParticleReplacement(locationParticles, field.shape[0], field.shape[1], totalRandomFactor=1)
                newLocationParticles.append(newParticle)
            elif particle is not None:
                newLocationParticles.append(particle)
            else:
                # Add new particle as replacement of dead particle
                newParticle = generateParticleReplacement(locationParticles, field.shape[0], field.shape[1])
                newLocationParticles.append(newParticle)
                # print("Replaced dead particle")
                pass
        locationParticles = newLocationParticles

        # Plot location particles
        for particle in locationParticles:
            dispField[int(particle[0]),int(particle[1])] = 150
    except:
        print("Failed processing particles")
        fails += 1
        continue

    try:
        cv2.imshow('vid',frame)
        cv2.imshow('lines',img)
        cv2.imshow('loc',dispField)
    except:
        print("Could not process frame and show result")
        break
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)